In [1]:
import os
import openai
import sys
sys.path.append('../..')
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
from langchain_text_splitters import HTMLHeaderTextSplitter
from pandas import read_csv
from unicodedata import normalize

### Initialize Data Frame

In [3]:
file_path="eskom_articles.csv"
df = read_csv(filepath_or_buffer=file_path, sep=";", encoding="utf-8")
df = df[~df["url"].str.contains('/en/')]  # drop english sites
df = df.dropna()

In [4]:
df.tail()

,name,category,url,text,html
572,bezpieczny-internet-i-wifi-w-szkole,cases,https://eskom.eu/bezpieczny-internet-i-wifi-w-...,Aruba – przyjazny bezprzewodowy dostęp do Inte...,"<div class=""entry-content"">\n<div class=""conta..."
573,pzm-autotur,cases,https://eskom.eu/pzm-autotur,Wyższa wydajność i nowe możliwości rozwoju\n„S...,"<div class=""entry-content"">\n<div class=""conta..."
574,fresh-world-international,cases,https://eskom.eu/fresh-world-international,ESKOM wspiera rozwój informatyczny firmy Fresh...,"<div class=""entry-content"">\n<div class=""conta..."
575,polsko-japonska-akademia-technik-komputerowych,cases,https://eskom.eu/polsko-japonska-akademia-tech...,Polsko-Japońska Akademia Technik Komputerowych...,"<div class=""entry-content"">\n<div class=""conta..."
576,transformacja-infrastruktury-it-i-poprawa-ciag...,cases,https://eskom.eu/case-study/transformacja-infr...,W miarę jak biznes staje się coraz bardziej cy...,"<div class=""entry-content"">\n<div class=""conta..."


In [43]:
shortdf = df.iloc[1:2]
shortdf

,name,category,url,text,html
1,jak-zapewnic-ochrone-dla-firmowych-danych-w-hy...,post,https://eskom.eu/blog/jak-zapewnic-ochrone-dla...,Dane traktowane są obecnie jako jeden z kluczo...,"<div class=""entry-content"">\n<h3>Dane traktowa..."


In [17]:
from langchain_text_splitters import HTMLHeaderTextSplitter
from pandas import read_csv
from unicodedata import normalize

In [ ]:
HEADERS_TO_SPLIT_ON = [('h1', 'Header 1'),
                       ('h2', 'Header 2'),
                       ('h3', 'Header 3'),
                       ('h4', 'Header 4'),
                       ('h5', 'Header 5'),
                       ('h6', 'Header 6'),
                       ('h7', 'Header 7')]

In [32]:
def fix_formatting(text):
    return normalize("NFKD", text).replace('\n', " ")

In [52]:
docs = []
for _, row in shortdf.iterrows():
    chunk = 1
    for doc in HTMLHeaderTextSplitter(headers_to_split_on=HEADERS_TO_SPLIT_ON).split_text(row["html"]):
        doc.page_content = fix_formatting(doc.page_content)
        doc.metadata.update({"source_url": fix_formatting(row["url"]),
                             "chunk": chunk,
                             "source_article": fix_formatting(row["name"]),
                             "category": fix_formatting(row["category"])})
        docs.append(doc)
        chunk += 1

In [54]:
docs[1].metadata

{'Header 3': 'Dane traktowane są\xa0obecnie jako jeden z\xa0kluczowych, jeśli nie\xa0najważniejszy organizacyjny zasób. Biznes po\xa0prostu zależy od\xa0danych. Niestety dostrzegli to\xa0także cyberprzestępcy, którzy\xa0zwielokrotnili próby ich wykradzenia albo\xa0przynajmniej zaszyfrowania. Można być pewnym, że\xa0wykorzystają każdą, nawet najmniejszą pozostawioną im\xa0lukę w\xa0systemie bezpieczeństwa, ponieważ wiedzą, że\xa0można na\xa0tym dobrze zarobić. Właśnie dlatego ochrona danych spędza sen z\xa0powiek już nie\xa0tylko\xa0menedżerom ds.\xa0bezpieczeństwa czy\xa0szefom działów informatyki, ale\xa0praktycznie całej wyższej kadrze kierowniczej w\xa0firmach i\xa0instytucjach.',
 'source_url': 'https://eskom.eu/blog/jak-zapewnic-ochrone-dla-firmowych-danych-w-hybrydowym-wielochmurowym-swiecie-biznesowego-it',
 'chunk': 2,
 'source_article': 'jak-zapewnic-ochrone-dla-firmowych-danych-w-hybrydowym-wielochmurowym-swiecie-biznesowego-it',
 'category': 'post'}